In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder#
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# Loading our MNIST dataset
data = load_digits()
X = data['data']
y = data['target']

# one-hot encoding
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1,1)).toarray()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X/16, y, test_size=0.3)

In [ ]:
def data_visualization(data, target, index):
    plt.figure(figsize=(8, 8))
    plt.imshow(data[index].reshape(8, 8), cmap='gray')
    plt.title(target[index])
    plt.axis('off')
    plt.show() 

# Visualizing the data
#data_visualization(X, y, 0)

In [ ]:
def data_generator(input_data, target_data, minibatch_size):
    # Get the total number of samples in the dataset
    num_samples = len(input_data)
    
    # Create an array of indices corresponding to the samples
    indices = np.arange(num_samples)
    # Shuffle the indices to randomize the order of samples
    np.random.shuffle(indices)

    # Iterate over the shuffled indices to yield mini-batches
    for start_idx in range(0, num_samples - minibatch_size + 1, minibatch_size):
        # Select a subset of indices for the current mini-batch
        excerpt = indices[start_idx:start_idx + minibatch_size]
        
        # Yield the corresponding input and target data for the mini-batch
        yield input_data[excerpt], target_data[excerpt]

In [ ]:
# Sigmoid activation function
class Sigmoid:
    def __init__(self):
        pass

    def call(self, z):
        result = []
        for y in z:
            for x in y:
                if x >= 0:
                    result.append(1. / ( 1. + np.exp(-x) ) )
                else:
                    result.append( np.exp(x) / ( 1. + np.exp(x) ) )
        result = np.array(result).reshape(z.shape)
        return result
    

    def gradient(self, x):
        sigmoid_x = self.call(x)
        gradient_x = sigmoid_x * (1 - sigmoid_x)
        return gradient_x

In [ ]:
# Cross-Entropy Loss function
class CrossEntropyLoss:
    def __init__(self):
        self.probs = None
        self.targets = None

    def forward(self, prediction, true_target):
        self.probs = prediction.T
        self.targets = true_target
        epsilon = 1e-15  # Small constant to avoid taking the log of zero

        prediction = np.clip(prediction, epsilon, 1 - epsilon)

        # Calculate the cross-entropy loss
        
        loss = -np.sum(self.targets * np.log(self.probs + epsilon))

        # Normalize the loss by the number of samples
        loss /= self.targets.shape[0]

        return loss

    def backward(self):
        # Calculate the gradient of the cross-entropy loss
        d_loss = self.probs - self.targets

        return d_loss

In [ ]:
# Softmax activation function
class Softmax:
    def __init__(self):
        pass

    def call(self, x):
        exp_x = np.exp(x - np.max(x))  # Subtracting the maximum value for numerical stability
        return exp_x / exp_x.sum(axis=-1, keepdims=True)

    def gradient(self, x):
        probabilities = self.call(x)
        return probabilities


In [ ]:
class MLP_Layer:
    def __init__(self, n_neurons, activation_function, index):
        # Initialize the MLP layer with the number of neurons, activation function, and index.
        self.neurons = n_neurons  # Number of neurons in the layer
        self.activation_function = activation_function  # Activation function to be used
        self.index = index  # Index of the layer
        self.activation = 0  # Defines the output of the layer after running through the activation function
        self.preactivation = 0  # Defines the input of the layer before running through the activation function

    def init_weight(self, inputs, neurons):
        # Initialize the weights and bias for the layer.
        self.weights = np.random.normal(loc=0.0, scale=10, size=(neurons, inputs))
        self.bias = np.zeros((neurons, 1))

    def Neuron_activation(self, inputs):
        # Compute the activation of the layer's neurons.
        try:
            self.preactivation = np.dot(self.weights, inputs) + self.bias.ravel()
        except:
            self.preactivation = np.dot(self.weights, inputs) + self.bias
        self.activation = self.activation_function.call(self.preactivation)
        return self.activation

    def activation_gradient(self):
        # Calculate the gradient of the activation function for backpropagation.
        return self.activation_function.gradient(self.activation)


In [ ]:
class MLP_print:
    def __init__(self, shapes, activation_functions, lossFunction):
        self.shape = shapes
        self.activation_functions = activation_functions
        self.layers = list()
        self.loss = lossFunction
        self.losses = list()
        self.accuracies = list()  # Store accuracies

        j = 0

        for shape in shapes:
            self.layers.append(MLP_Layer(shape, activation_functions[j], j))
            j += 1

        for i in range(len(self.layers)):
            if i == 0:
                self.layers[i].init_weight(X.shape[1], self.layers[i].neurons)
            else:
                self.layers[i].init_weight(self.layers[i - 1].neurons, self.layers[i].neurons)
        """ 
    def forward(self, inputs):
        a = inputs.T
        for layer in self.layers:
        a = layer.Neuron_activation(a)  
         """

    def compute_loss(self, result, targets):
        loss = self.loss.forward(result, targets)
        #print("Loss computed.")
        return loss

    def backward(self, inputs, targets):
        # Initialize lists to store weight and bias gradients for each layer
        delta_w = [0 for _ in range(len(self.layers))]
        delta_b = [0 for _ in range(len(self.layers))]
    
        # Calculate the error in the output layer

        error_o = (self.layers[-1].preactivation - targets.T)
    
        # Iterate through the layers in reverse order to calculate gradients
        for i in reversed(range(len(self.layers) - 1)):
            # Calculate the error for the current layer
            error_i = np.multiply(self.layers[i + 1].weights.T.dot(error_o), self.layers[i].activation_function.gradient(self.layers[i].preactivation))
        
            # Calculate the weight gradients for the current layer
            delta_w[i + 1] = error_o.dot(self.layers[i].activation.T) / len(targets)
        
            # Calculate the bias gradients for the current layer
            delta_b[i + 1] = np.sum(error_o, axis=1, keepdims=True) / len(targets)
        
            # Update the error for the next layer in the iteration
            error_o = error_i
        
            # Print progress information
            #print("Backward pass for layer", self.layers[i].index, "completed.")
            #print("Delta Weights for layer", self.layers[i].index, ":\n", delta_w[i + 1])
            #print("Delta Biases for layer", self.layers[i].index, ":\n", delta_b[i + 1])

        # Calculate the gradients for the input layer
        try:
            delta_w[0] = error_o.dot(inputs) / len(targets)
            delta_b[0] = np.sum(error_o, axis=1, keepdims=True) / len(targets)
        except:
            delta_w[0] = error_o.dot(inputs.T)
            delta_b[0] = np.sum(error_o, axis=1, keepdims=True) / len(targets)
        
        #print("Backward pass completed.")
        return delta_w, delta_b
    def calculate_accuracy(self, result, targets):
        # Compare predicted results to true labels and calculate accuracy
        accuracy = np.mean(np.argmax(result, axis=0) == np.argmax(targets, axis=0))
        return accuracy

    def update_weights_and_biases(self, delta_w, delta_b, learning_rate):
        for i in range(len(self.layers)):
            self.layers[i].weights -= learning_rate * delta_w[i]
            self.layers[i].bias -= learning_rate * delta_b[i]
            #print("Weights and biases for layer", self.layers[i].index, "updated.")
            #print("Updated Weights for layer", self.layers[i].index, ":\n", self.layers[i].weights)
            #print("Updated Biases for layer", self.layers[i].index, ":\n", self.layers[i].bias)
        #print("Weights and biases updated.")

    def train(self, inputs, targets, learning_rate, epochs, batch_size):
        for epoch in range(epochs):
            batches = data_generator(inputs, targets, batch_size)
            for batch in batches:
                batch_x, batch_y = batch
                result = self.predict(batch_x)
            delta_w, delta_b = self.backward(batch_x, batch_y)
            self.update_weights_and_biases(delta_w, delta_b, learning_rate)
            self.losses.append(self.compute_loss(result, batch_y))

                # Calculate and store accuracy for each batch
            accuracy = self.calculate_accuracy(result.T, batch_y)*100
            self.accuracies.append(accuracy)

            #if (epoch+1) % 1000 == 0:
                #print("Epoch: ", epoch + 1, " Loss: ", self.losses[-1])
            #elif epoch == 1:
                #print("Epoch: ", epoch, " Loss: ", self.losses[-1])

    def predict(self, X):

        a = X.T
        for layer in self.layers:
            a = layer.Neuron_activation(a)
        #print("Prediction pass for layer", layer.index, "completed.")
        #print("Prediction completed.")
        return a


In [ ]:
# Define the model and its architecture
model = MLP_print([64,128,69,32,32,16, 10], [Sigmoid(),Sigmoid(),Sigmoid(),Sigmoid(), Sigmoid(), Sigmoid(), Softmax()], CrossEntropyLoss())

epoch_numbers = 1000

# Train the model
model.train(X_train, y_train, 0.01, epoch_numbers, 32)

# Plot the loss curve
plt.figure(figsize=(20, 5))
plt.plot(model.losses, label="Loss")
plt.plot(model.accuracies, label="Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.show()

print("Training Accuracy: ", np.mean(np.argmax(model.predict(X_train), axis=0) == np.argmax(y_train, axis=1)))
data_visualization(X_test, y_test, 0)
temp = model.predict(X_test[0])
data_visualization(X_test, temp, 0)
print(temp)

In [ ]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Create an instance of the MLP class
shape = [2, 1] # Define the network architecture
activation_functions = [Sigmoid(), Sigmoid()]
lossFunction = CrossEntropyLoss()
mlp = MLP_print(shape, activation_functions, lossFunction)

# Train the MLP on the dataset
learning_rate = 0.1
epochs = 10
batch_size = 2
mlp.train(X, y, learning_rate, epochs, batch_size)

# Make predictions
#
predictions = mlp.predict(X[1])
print("Predictions:")
print(predictions)